# Scoring Function

In [166]:
def project_1_scoring(data):
    from sklearn.preprocessing import OneHotEncoder
    from copy import deepcopy
    from sklearn.linear_model import LogisticRegression
    import pickle
    import category_encoders as ce
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    # we deep copy original data
    X_train= data.copy()
    #replacing values in RevLineCr with Y and N
    X_train['RevLineCr']=X_train['RevLineCr'].replace('0','N')
    X_train['RevLineCr']=X_train['RevLineCr'].replace('1','Y')
    X_train['RevLineCr']=X_train['RevLineCr'].replace('T','Y')
    #replacing values in RevLineCr with null that have no context
    X_train['RevLineCr'] = X_train['RevLineCr'].apply(lambda x: x if x in ['Y', 'N'] else np.nan)

    
    X_train['RevLineCr'].fillna(X_train['RevLineCr'].mode()[0], inplace=True)

    #replacing values in LowDoc with Y and N
    X_train['LowDoc']=X_train['LowDoc'].replace('0','N')
    X_train['LowDoc']=X_train['LowDoc'].replace('S','Y')
    X_train['LowDoc']=X_train['LowDoc'].replace('A','N')
    
    X_train['LowDoc'] = X_train['LowDoc'].apply(lambda x: x if x in ['Y', 'N'] else np.nan)


    X_train['LowDoc'].fillna(X_train['LowDoc'].mode()[0], inplace=True)

    #extract index column 
    index=X_train[['index']]
    X_train.drop('index',inplace=True,axis=1,errors='ignore')
    cat=[]
    num=[]
    colu=[]
    for col in X_train.columns:
        if X_train[col].isna().any() == True:
            colu.append(col)
    num=[col for col in colu if (X_train[col].dtype != 'object') ] 
    cat=[col for col in colu if (X_train[col].dtype == 'object') ]
    #imputing null values
    for i in cat:
        X_train[i].fillna(X_train[i].mode().iloc[0],inplace=True)
    for i in num:
        X_train[i].fillna(X_train[i].median(),inplace=True)
    # Fearure engineering
    # Feature 1: Loan Size Category
    X_train['LoanSizeCategory'] = pd.cut(X_train['GrAppv'], bins=[0, 50000, 150000, 350000, 1000000, 5000000], labels=['Micro', 'Small', 'Medium', 'Large', 'Very Large'])

        # Feature 2: Job Creation Category
    X_train['JobCreationCategory'] = pd.cut(X_train['CreateJob'] + X_train['RetainedJob'], bins=[-1, 0, 5, 10, 20, 50, 100, float('inf')], labels=['None', 'Few', 'Some', 'Moderate', 'Many', 'A Lot', 'Massive'])

        # Feature 3: High-Risk Industry (NAICS codes starting with 72 are typically hospitality, considered higher risk)
    X_train['HighRiskIndustry'] = X_train['NAICS'].apply(lambda x: 1 if str(x).startswith('72') else 0)

        # Feature 4: Loan/Grant Ratio
    X_train['LoanGrantRatio'] = X_train['GrAppv'] / X_train['SBA_Appv']

        # Feature 5: Per Employee Investment
    X_train['PerEmployeeInvestment'] = X_train['GrAppv'] / (X_train['NoEmp'] + 1)  # Avoid division by zero



        # Feature 6: Business Age - New business flag (assuming NewExist codes: 1 for existing, 2 for new)
    X_train['NewBusiness'] = X_train['NewExist'].apply(lambda x: 1 if x == 2 else 0)

        # Feature 7: Urban Business - Flag for businesses in urban areas (assuming UrbanRural codes: 1 for urban, 2 for rural, 0 for undefined)
    X_train['UrbanBusiness'] = X_train['UrbanRural'].apply(lambda x: 1 if x == 1 else 0)

        # Feature 8: Franchise Flag - Flag for whether the business is a franchise (assuming FranchiseCode: 1 or 0 indicates not a franchise, any other value indicates a franchise)
    X_train['FranchiseFlag'] = X_train['FranchiseCode'].apply(lambda x: 0 if x == 1 or x == 0 else 1)

        # Feature 9: Expansion Plan - Flag businesses creating more jobs than retaining (CreateJob > RetainedJob)
    X_train['ExpansionPlan'] = X_train.apply(lambda x: 1 if x['CreateJob'] > x['RetainedJob'] else 0, axis=1)

        # Feature 10: Economic Sector - Extract broader economic sector from NAICS code (first two digits of NAICS code)
    X_train['EconomicSector'] = X_train['NAICS'].apply(lambda x: int(str(x)[:2]) if x > 0 else 0)

    for col in X_train.select_dtypes(include=['category']).columns:
        X_train[col] = X_train[col].astype('object')
    # Identify categorical variables with less than 10 unique values for one-hot encoding
    cat_vars_for_one_hot = [col for col in X_train.columns if (X_train[col].nunique() < 10) and (X_train[col].dtype == 'object')]

    # Now, for target and WOE encoding, identify the remaining categorical variables
    remaining_cat_vars = [col for col in X_train.columns if (X_train[col].dtype == 'object') and (col not in cat_vars_for_one_hot)]

    numeric=[col for col in X_train.columns if (X_train[col].dtype != 'object') ]

    #importing trained encoders/scalers
    with open('woe_encoder.pkl', 'rb') as file:
        loaded_woe_encoder = pickle.load(file)
    with open('standardscaler.pkl', 'rb') as file:
        loaded_scaler = pickle.load(file)
    with open('onehot_encoder.pkl', 'rb') as file:
        loaded_onehot_encoder = pickle.load(file)
    drops=['NewExist','CreateJob','RetainedJob','NAICS','GrAppv','SBA_Appv']
    res = [item for item in numeric if item not in drops]
    
    X_train = X_train.join(loaded_woe_encoder.transform(X_train[remaining_cat_vars]).add_suffix('_woe'))
    
    X_train[res] = loaded_scaler.transform(X_train[res])
    
    X_train.drop(columns=remaining_cat_vars,axis=1,inplace=True,errors='ignore')
    X_train.drop(columns=drops,axis=1,inplace=True,errors='ignore')

    X_train_ohe = loaded_onehot_encoder.transform(X_train[cat_vars_for_one_hot])
    # Convert to DataFrame and name columns
    columns = loaded_onehot_encoder.get_feature_names_out(cat_vars_for_one_hot)
    X_train_ohe = pd.DataFrame(X_train_ohe, columns=columns, index=X_train.index)

    X_train = pd.concat([X_train.drop(columns=cat_vars_for_one_hot), X_train_ohe], axis=1)
    #importing trained logreg model
    with open('logistic_model.pkl', 'rb') as file:
        loaded_model = pickle.load(file)
    y_pred_proba = loaded_model.predict_proba(X_train)
    y_pred = (y_pred_proba[:,1] > 0.54).astype(np.int16)
    d = {"index":index.to_numpy().ravel(),
             "label":y_pred,
             "probability_0":y_pred_proba[:,0],
             "probability_1":y_pred_proba[:,1]}

    return pd.DataFrame(d)

In [168]:
import pandas as pd
data = pd.read_csv('SBA_loans_project_1_holdout_students_valid.csv')
project_1_scoring(data)

,index,label,probability_0,probability_1
0,0,0,0.935179,0.064821
1,1,0,0.803215,0.196785
2,2,0,0.716843,0.283157
3,3,0,0.890897,0.109103
4,4,1,0.428825,0.571175
...,...,...,...,...
98904,98904,1,0.411246,0.588754
98905,98905,0,0.790809,0.209191
98906,98906,1,0.250261,0.749739
98907,98907,0,0.498160,0.501840


# Kaggle Submission document

In [144]:
x=project_1_scoring(data)
x=x[['index','probability_1']]

(98909, 31)


In [145]:
x.rename(columns={'index': 'ID'}, inplace=True)

In [146]:
x

,ID,probability_1
0,0,0.064821
1,1,0.196785
2,2,0.283157
3,3,0.109103
4,4,0.571175
...,...,...
98904,98904,0.588754
98905,98905,0.209191
98906,98906,0.749739
98907,98907,0.501840


In [151]:
x.to_csv('sub.csv', index=False)